## Reconstructing images from few pixels

In [ ]:
from pit import *

# packages for reading the image
import scipy.ndimage as spimg
from scipy.misc import imsave
import glob
import os


### ===============================================
### ---------- set image folder -------------------
folder = "/Users/Martin/Pictures/Camera Roll/"
### -----------------------------------------------

# all jpg files in folder
list_camera_pics = glob.glob(folder+'*.jpg') 
# file name of latest one
pic_file = max(list_camera_pics, key=os.path.getctime) 

### -----------------------------------------------
### ---- OR choose picture from the repository ----
pic_file = "./pics/marie_curie.jpg"
### ===============================================


# read image
Xorig = spimg.imread(pic_file, flatten=True, mode='L')

# pywt can change the size -> easy fix by transforming back and forth
x = pywt.wavedec2(Xorig, wavelet='db1', level=1)
Xorig = pywt.waverec2(x, wavelet='db1')

# picture size
shape = Xorig.shape
n = np.prod(shape)

# choice of WT parameters
L = 3
amp = np.linspace(1,.5,L)
amp = np.kron(amp, np.ones(3) )
amp = np.insert(amp,0, 10 ) #prepend (position 0)

### A compression test 

In [ ]:
## (not necessary for the rest)
# choose basis for compression              
T = WT(shape, wavelet='db6',level=L, amplify=amp)

# hard TO
s = int( np.prod(shape)/20 )
TO = hardTO(s)

# compression test
cXorig = compress(T,TO,Xorig)

print("Original image:")
pltPic(Xorig)

print("Compressed image (down to {} coefficients out of {}):".format(s,n) )
pltPic(cXorig)

### Set number of pixels to keep

In [ ]:
# set relative number of pixels
p = .05

# total number of pixels
m = int(p*n)
print("Total number of pixels: {}k".format(int(np.prod(Xorig.shape)/1000)))
print("Number of remaining pixels: {}k".format( int(m/1000) ))

# generate subsampled picture
mask = getRandMask(n, m)
Xsub = np.zeros(shape)
Xsub.flat[mask] = Xorig.flat[mask]
pltPic(Xsub)

### Choose parameters and run reconstruction

In [ ]:
steps = 10 #parameter for nr. of steps in fixed basis
N = 6 #nr. of switching basises: wt and dct

# soft thresholding parameters

# # adjusted to p = .2
# taus_wt = np.linspace(12,4,N)
# taus_dct = np.linspace(8,3.25,N)

# # adjusted to p = .1
# taus_wt = np.linspace(15,7,N)
# taus_dct = np.linspace(10,5,N)

# adjusted to p = .05
taus_wt = np.linspace(25,6,N)
taus_dct = np.linspace(15,9,N)

# # adjusted to p = .02
# taus_wt = np.linspace(24,16,N)
# taus_dct = np.linspace(19,17.5,N)


# set initial guess for the reconstruction
if 'Xrec' not in locals():
    Xrec = Xsub
    
# Xrec = Xsub
    
# set transformations
dct = DCT(shape)
wt = WT(shape, wavelet='db6',level=L, amplify=amp)

for j in range(N):
#     Xrec = Xsub
    thOp = softTO(taus_dct[j])
#     print('DCT (tau = {}):'.format(taus_dct[j]), end = ' ')
    Xrec=estimate(dct, thOp, mask, Xsub, n_steps=steps, Xorig = Xorig, X0=Xrec)
    thOp = softTO(taus_wt[j])
#     print('WT (tau = {:.2f}):'.format(taus_wt[j]), end = ' ')    
    Xrec=estimate(wt, thOp, mask, Xsub, n_steps=3*steps, Xorig = Xorig, X0 = Xrec)

pltPic(Xrec)

### Compare to original picture

In [ ]:
pltPic(Xorig)

## Export images

In [ ]:
# make black pixels in the masked picture Xsub white
Xsub2 = Xsub.copy()
Xsub2[Xsub2==0.0] = Xsub2.max()

imsave("./pics/marie_curie_masked.jpg", Xsub2)
imsave("./pics/marie_curie_rec.jpg", Xrec)